In [ ]:
#TODO siehe Datein unten
#TODO Distanz etc. bei Sammelmarker (siehe ChatGPT Vorschläge) hereinnehmen
#TODO Start Bildschirm siehe ChatGPT
#TODO Legende Heatmap heraus und Sammelmarker Erklärung auch
#TODO Name Briefe Heatmap zu Heatmap
#TODO komplett anschauen und verstehen
#TODO Heatmap mehr in die Breite?
#TODO andere Karte im Hintergrund

In [2]:
import pandas as pd

df = pd.read_csv("rom_korr_full_website_coords.csv")

# Die ersten 5 Zeilen anzeigen, um sicherzustellen, dass die Daten korrekt geladen wurden
display(df)
#print(df_test.head(5))

,Date,Sender,Recipient,Place of Dispatch,Place of Destination,Dispatch_GeoNames,Destination_GeoNames,link,Dispatch_Lat,Dispatch_Lon,Destination_Lat,Destination_Lon,Distance_km
0,"Donnerstag, 6. Januar 1791",August Wilhelm von Schlegel,Christian Gottlob Heyne,Göttingen,Göttingen,https://www.geonames.org/2918632,https://www.geonames.org/2918632,https://briefe-der-romantik.de/letters/view/1?...,51.53443,9.93228,51.53443,9.93228,0.000000
1,"Mittwoch, 20. September 1797",August Wilhelm von Schlegel,Christian Gottlob Heyne,Jena,Göttingen,https://www.geonames.org/2895044,https://www.geonames.org/2918632,https://briefe-der-romantik.de/letters/view/2?...,50.92878,11.58990,51.53443,9.93228,133.955266
2,"Samstag, 26. Mai 1798",August Wilhelm von Schlegel,Georg Joachim Göschen,Berlin,Leipzig,https://www.geonames.org/2950159,https://www.geonames.org/2879139,https://briefe-der-romantik.de/letters/view/3?...,52.52437,13.41053,51.33962,12.37129,149.951785
3,"Mittwoch, 31. Oktober 1798",August Wilhelm von Schlegel,Georg Joachim Göschen,Jena,Leipzig,https://www.geonames.org/2895044,https://www.geonames.org/2879139,https://briefe-der-romantik.de/letters/view/4?...,50.92878,11.58990,51.33962,12.37129,71.276442
4,[Mitte August 1801],Sophie Bernhardi,August Wilhelm von Schlegel,Berlin,Jena,https://www.geonames.org/2950159,https://www.geonames.org/2895044,https://briefe-der-romantik.de/letters/view/5?...,52.52437,13.41053,50.92878,11.58990,217.565424
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4383,[vor dem 16.04.1795],Gottfried Philipp Michaelis,Caroline von Schelling,"Harburg, Elbe",Braunschweig,https://www.geonames.org/2910685,https://www.geonames.org/2945024,https://briefe-der-romantik.de/letters/view/19...,53.46057,9.98388,52.26594,10.52673,137.877321
4384,vor dem 10. Dezember 1801,Johann Friedrich Gottlieb Unger,Ludwig Tieck,Berlin,Dresden,https://www.geonames.org/2950159,https://www.geonames.org/2935022,https://briefe-der-romantik.de/letters/view/20...,52.52437,13.41053,51.05089,13.73832,165.496646
4385,"Mittwoch, 9. Dezember 1801",Ludwig Tieck,Carl Friedrich Ernst Frommann,Dresden,Jena,https://www.geonames.org/2935022,https://www.geonames.org/2895044,https://briefe-der-romantik.de/letters/view/20...,51.05089,13.73832,50.92878,11.58990,151.452012
4386,"Donnerstag, 10. Dezember 1801",Ludwig Tieck,Johann Friedrich Gottlieb Unger,Dresden,Berlin,https://www.geonames.org/2935022,https://www.geonames.org/2950159,https://briefe-der-romantik.de/letters/view/20...,51.05089,13.73832,52.52437,13.41053,165.496646


In [9]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster, HeatMap
from folium.features import GeoJson, GeoJsonPopup
from folium import Map, Marker
from folium.map import LayerControl, FeatureGroup
from folium import plugins
from folium import Element

##########################################
# FUNKTION: LEGENDE ERSTELLEN
##########################################
def add_legend(map_obj):
    """
    Adds a legend to the folium map
    """
    legend_html = '''
     <div style="
     position: fixed; 
     bottom: 50px; left: 50px; width: 250px; height: auto; 
     background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
     padding: 10px;">
     <b>Legende:</b> <br>
     &nbsp; <i style="color:red;" class="fa fa-map-marker"></i> Absender <br>
     &nbsp; <i style="color:#357EDD;" class="fa fa-map-marker"></i> Empfänger <br>
     &nbsp; <i style="color:purple;" class="fa fa-map-marker"></i> Sammelmarker <br>
     &nbsp; <span style="color:green; font-size: 20px;">&#x2501;</span> Distanzlinie (grüne Linie, km) <br>
     <hr>
     <small>
     © <a href="https://briefe-der-romantik.de" target="_blank">briefe-der-romantik.de</a> <br>
     </small></div>
    '''
    map_obj.get_root().html.add_child(Element(legend_html))

##########################################
# FUNKTION: SAMMELPOPUP FÜR VIELE EINTRÄGE
##########################################
def create_sammelpopup_html(group_df):
    items_html = []
    for idx, row in group_df.iterrows():
        sender = row.get('Sender', '?')
        recipient = row.get('Recipient', '?')
        date_str = row.get('Date', '?')
        link_str = row.get('link', '#')
        
        # Absender-/Empfänger-Kürzel
        ort_typ = 'Abs.' if row['type'] == 'dispatch' else 'Empf.'
        
        # Metadaten aus marker_df holen
        absendeort = row.get('Place_of_Dispatch', '?')
        empfangsort = row.get('Place_of_Destination', '?')

        # Distanz - falls nicht numerisch, einfach als String anzeigen
        dist_str = row.get('Distance_km', '?')
        try:
            # Falls du es als float vorliegen hast, formatieren:
            dist_str = f"{float(dist_str):.1f} km"
        except:
            pass  # Falls es z.B. ein „?“ ist, nichts tun
        
        heading = f"{ort_typ}: {sender} → {recipient}"
        items_html.append(
            f"<li>"
            f"<b>{heading}</b><br>"
            f"Absendeort: {absendeort}<br>"
            f"Empfangsort: {empfangsort}<br>"
            f"Distanz: {dist_str}<br>"
            f"Datum: {date_str} [<a href='{link_str}' target='_blank'>Original</a>]"
            f"</li>"
        )
    
    items_joined = "\n".join(items_html)
    
    # In einen scrollbaren Div packen (falls sehr lang):
    html = f"""
    <h4>Sammelmarker ({len(group_df)} Einträge)</h4>
    <div style="max-height:250px; overflow:auto;">
    <ul>
    {items_joined}
    </ul>
    </div>
    """
    return html



##########################################
# FUNKTION: GEOJSON-LINIE (Popups)
##########################################
def add_line_geojson(map_obj, lat1, lon1, lat2, lon2, popup_html):
    """
    Erzeugt ein GeoJSON-LineString-Feature, das klickbar ist
    und ein Popup anzeigt. [LON, LAT]-Reihenfolge beachten.
    """
    geojson_feature = {
        "type": "Feature",
        "properties": {
            "popup": popup_html
        },
        "geometry": {
            "type": "LineString",
            "coordinates": [
                [lon1, lat1],
                [lon2, lat2]
            ]
        }
    }

    popup = GeoJsonPopup(fields=["popup"], labels=False, max_width=300)

    def style_function(feature):
        return {
            "color": "green",
            "weight": 2,
            "opacity": 0.4
        }

    geojson_obj = GeoJson(
        data=geojson_feature,
        style_function=style_function,
        name="Line",
        popup=popup,
        tooltip="Klicken für mehr Info"
    )

    geojson_obj.add_to(map_obj)

##########################################
# FUNKTION: HEATMAP-LAYER
##########################################
def add_heatmap_layer(df, map_obj, layer_name="Heatmap"):
    """
    Legt einen Heatmap-Layer über alle Dispatch- und Destination-Koordinaten.
    """
    heat_data = []

    for idx, row in df.iterrows():
        if pd.notnull(row['Dispatch_Lat']) and pd.notnull(row['Dispatch_Lon']):
            heat_data.append([row['Dispatch_Lat'], row['Dispatch_Lon']])
        if pd.notnull(row['Destination_Lat']) and pd.notnull(row['Destination_Lon']):
            heat_data.append([row['Destination_Lat'], row['Destination_Lon']])

    if len(heat_data) > 0:
        heat_layer = HeatMap(
            heat_data, 
            name=layer_name, 
            radius=30, 
            blur=10, 
            min_opacity=0.4,
            show=False            # <-- sorgt dafür, dass der Layer "aus" ist
        )
        heat_layer.add_to(map_obj)


##########################################
# HAUPTFUNKTION: KARTE GENERIEREN
##########################################
def generate_map(df):
    """
    Erzeugt die Folium-Map mit:
      - Heatmap
      - Linien
      - Markern:
         * Falls >100 Einträge an (lat,lon): Sammelmarker (lila).
         * Sonst Spiderfy (MarkerCluster) mit roten/ blauen Markern
           (rot = Absender, blau = Empfänger).
    """
    # ------------------
    # 1) Kartenmittelpunkt berechnen
    valid_lat = df['Dispatch_Lat'].dropna().tolist() + df['Destination_Lat'].dropna().tolist()
    valid_lon = df['Dispatch_Lon'].dropna().tolist() + df['Destination_Lon'].dropna().tolist()
    if len(valid_lat) > 0 and len(valid_lon) > 0:
        avg_lat = sum(valid_lat) / len(valid_lat)
        avg_lon = sum(valid_lon) / len(valid_lon)
    else:
        avg_lat, avg_lon = 50.0, 10.0

    my_map = folium.Map(location=[avg_lat, avg_lon], zoom_start=6)

    # ------------------
    # 2) Heatmap-Layer anlegen
    add_heatmap_layer(df, my_map, layer_name="Heatmap")

    # ------------------
    # 3) FeatureGroup für Marker und Linien (so kann man manuell toggeln)
    marker_line_group = folium.FeatureGroup(name="Marker & Linien")
    marker_line_group.add_to(my_map)

    # 3a) MarkerCluster für Spiderfy
    cluster = MarkerCluster(
        options={
            "spiderfyOnEveryZoom": True,
            "showCoverageOnHover": True
        }
    )
    cluster.add_to(marker_line_group)

    # ------------------
    # 4) LINIEN hinzufügen (wie gehabt, pro Zeile)
    for idx, row in df.iterrows():
        lat1 = row['Dispatch_Lat']
        lon1 = row['Dispatch_Lon']
        lat2 = row['Destination_Lat']
        lon2 = row['Destination_Lon']

        if pd.notnull(lat1) and pd.notnull(lon1) and pd.notnull(lat2) and pd.notnull(lon2):
            sender = row.get("Sender", "?")
            recipient = row.get("Recipient", "?")
            place_disp = row.get("Place of Dispatch","?")
            place_dest = row.get("Place of Destination","?")
            date_str = row.get("Date","?")
            link_str = row.get("link","#")
            distance = row.get("Distance_km")
            dist_info = ""
            if pd.notnull(distance):
                dist_info = f"Distanz: ~{distance:.1f} km<br/>"

            heading = f"{sender} → {recipient}"
            line_popup_html = f"""
            <b>{heading}</b><br/>
            <i>Absendeort:</i> {place_disp}<br/>
            <i>Empfangsort:</i> {place_dest}<br/>
            <i>Datum:</i> {date_str}<br/>
            {dist_info}
            <i>Link:</i> <a href="{link_str}" target="_blank">Original</a>
            """
            add_line_geojson(marker_line_group, lat1, lon1, lat2, lon2, line_popup_html)

    # ------------------
    # 5) GEMEINSAME MARKER-Basis bauen (Dispatch und Destination vereinen).
    #    Wir erzeugen einen Hilfs-DataFrame "marker_df" mit einer Spalte "type",
    #    damit wir wissen, ob es sich um Absender (dispatch) oder Empfänger (destination) handelt.
    marker_rows = []
    for idx, row in df.iterrows():
        # Absender
        if pd.notnull(row['Dispatch_Lat']) and pd.notnull(row['Dispatch_Lon']):
            marker_rows.append({
                'Lat': row['Dispatch_Lat'],
                'Lon': row['Dispatch_Lon'],
                'type': 'dispatch',  # Kennzeichnung: Absender
                'Sender': row.get("Sender","?"),
                'Recipient': row.get("Recipient","?"),
                'Date': row.get("Date","?"),
                'link': row.get("link","#"),
                # NEU: Beide Orte und Distanz in die Marker-Daten
                'Place_of_Dispatch': row.get("Place of Dispatch", "?"),
                'Place_of_Destination': row.get("Place of Destination", "?"),
                'Distance_km': row.get("Distance_km", "?")
            })
        # Empfänger
        if pd.notnull(row['Destination_Lat']) and pd.notnull(row['Destination_Lon']):
            marker_rows.append({
                'Lat': row['Destination_Lat'],
                'Lon': row['Destination_Lon'],
                'type': 'destination',  # Kennzeichnung: Empfänger
                'Sender': row.get("Sender","?"),
                'Recipient': row.get("Recipient","?"),
                'Date': row.get("Date","?"),
                'link': row.get("link","#"),
                # NEU: Beide Orte und Distanz in die Marker-Daten
                'Place_of_Dispatch': row.get("Place of Dispatch", "?"),
                'Place_of_Destination': row.get("Place of Destination", "?"),
                'Distance_km': row.get("Distance_km", "?")
            })

    marker_df = pd.DataFrame(marker_rows)

    # ------------------
    # 5a) Gruppieren nach (Lat, Lon). => Dann haben wir Absender+Empfänger vereint.
    grouped = marker_df.groupby(['Lat', 'Lon'])

    # ------------------
    # 6) Für jede Gruppe entscheiden: Sammelmarker oder einzelne Spiderfy-Marker
    for (lat, lon), group_df in grouped:
        count_entries = len(group_df)
        
        if count_entries > 100:
            # => Sammelmarker (lila) bleibt wie gehabt
            sammel_popup = create_sammelpopup_html(group_df)
            folium.Marker(
                location=[lat, lon],
                popup=folium.Popup(sammel_popup, max_width=300),
                icon=folium.Icon(color="purple", icon="info-sign", prefix="fa")
            ).add_to(marker_line_group)
            
        else:
            # => einzelne Marker (rot bei dispatch, blau bei destination)
            for idx2, row2 in group_df.iterrows():
                m_type = row2['type']
                color = 'red' if m_type == 'dispatch' else 'blue'
                
                sender = row2.get("Sender", "?")
                recipient = row2.get("Recipient", "?")
                date_str = row2.get("Date", "?")
                link_str = row2.get("link", "#")
                
                # NEU: Wir holen immer beide Ortsnamen und ggf. Distanz
                place_disp = row2.get("PlaceDisp", "?")
                place_dest = row2.get("PlaceDest", "?")
                distance_val = row2.get("Distance_km", None)
                
                # NEU: Distance-Info vorbereiten
                if pd.notnull(distance_val):
                    dist_info = f"<i>Distanz:</i> ~{distance_val:.1f} km<br/>"
                else:
                    dist_info = ""
                
                heading = f"{sender} → {recipient}"
                
                # NEU: Popup immer mit beiden Orten + Distanz
                popup_text = (
                    f"<b>{heading}</b><br/>"
                    f"<i>Absendeort:</i> {place_disp}<br/>"
                    f"<i>Empfangsort:</i> {place_dest}<br/>"
                    f"<i>Datum:</i> {date_str}<br/>"
                    f"{dist_info}"
                    f"<i>Link:</i> <a href='{link_str}' target='_blank'>Original</a>"
                )

                folium.Marker(
                    location=[lat, lon],
                    popup=folium.Popup(popup_text, max_width=300),
                    icon=folium.Icon(color=color, icon='info-sign')
                ).add_to(cluster)

    # LayerControl und Legende hinzufügen
    folium.LayerControl().add_to(my_map)
    add_legend(my_map)

    # --------------------------------------------
    #  GLOBALER "MODAL"-POPUP
    # --------------------------------------------
    global_popup_html = """
    <div id="globalPopup" 
         style="position: fixed; top: 0; left: 0; width: 100%; height: 100%;
                background-color: rgba(0,0,0,0.5); z-index:9999; 
                display: flex; justify-content: center; align-items: center;">
      
      <div style="background-color: white; padding: 20px; border-radius: 10px;
                  max-width: 600px;">
        <h3>Herzlich Willkommen!</h3>
        <p>
          Auf dieser Karte finden Sie Brief-Korrespondenzen aus der Romantik. Grundlage der Karte ist der Korpus <a href="https://briefe-der-romantik.de" target="_blank">briefe-der-romantik.de</a>. 
        </p>
        <p>
          Beim Zoom in die Karte werden einzelne Briefe sichtbar. Klicken Sie darauf, können Sie Daten zu den Briefen einsehen, ebenso findet sich ein Link zum Originaltext. Bei Orten mit mehr als 100 Korrespondenzen, z. B. Jena oder Zürich, ist ein lila Sammelmarker eingefügt. Öffnet man ihn, finden sich alle Korrespondenzen dieses Ortes als Liste aufgeführt. Zwischen Absende- oder Empfangsort wurde die Distanz bestimmt und durch die Linie dargestellt.  Weiterhin kann rechts oben in der Karte eine Heatmap aktiviert werden, die die Intensität der Korrespondenzzahl einzelner Orte und Gegenden visualisiert.
        </p>
        <p>
          Bei Fragen und Anregungen einfach an robert.metzner@posteo.de schreiben. Danke!
        </p>
        <button onclick="document.getElementById('globalPopup').style.display='none'">
            Schließen
        </button>
      </div>
    </div>
    """

    # In das HTML des Map-Objekts einbinden
    my_map.get_root().html.add_child(Element(global_popup_html))
    
    return my_map


##########################################
# HAUPTPROGRAMM
##########################################
if __name__ == "__main__":
    # 1) Daten laden
    df = pd.read_csv("rom_korr_full_website_coords.csv")


    # 3) Karte erzeugen
    my_map = generate_map(df)

    # 4) Karte speichern
    my_map.save("rom_korr_080125_2.html")
    print("Karte gespeichert")


Karte gespeichert
